### PyAgrum Demo

Import the PyAgrum package

In [1]:
import pyAgrum as gum

Create an empty BN network

In [2]:
bn=gum.BayesNet('Demo')
print(bn)

BN{nodes: 0, arcs: 0, domainSize: 1, dim: 0, mem: 0o}


### Create the variables

pyAgrum provides 4 types of variables :

- LabelizedVariable
- RangeVariable
- IntegerVariable
- DiscretizedVariable


In [3]:
s=bn.add(gum.LabelizedVariable('Smoking','Smoking', ['no','yes']))
c=bn.add(gum.LabelizedVariable('Lung_Cancer','Lung_Cancer', ['no','yes']))
b=bn.add(gum.LabelizedVariable('Bronchitis','Bronchitis', ['no','yes']))
d=bn.add(gum.LabelizedVariable('Dyspnoea','Dyspnoea', ['no','yes']))
print(s)
print(c)
print(b)
print(d)
print(bn)


0
1
2
3
BN{nodes: 4, arcs: 0, domainSize: 16, dim: 4, mem: 64o}


### Create arcs


In [4]:
for link in [(s,c),(s,b),(b,d),(c,d)]:
    bn.addArc(*link)
print(bn)

BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}


pyAgrum provides tools to display <tt>bn</tt> in more user-frendly fashions. <br/>Notably, <tt>pyAgrum.lib</tt> is a set of tools written in pyAgrum to help using aGrUM in python. <tt>pyAgrum.lib.notebook</tt> adds dedicated functions for iPython notebook.

In [5]:
import pyAgrum.lib.notebook as gnb
bn

(pyAgrum.BayesNet<double>@00000252AF5220C0) BN{nodes: 4, arcs: 4, domainSize: 16, dim: 9, mem: 144o}

### Create the probability tables

Once the network topology is constructed, we must initialize the conditional probability tables (CPT) distributions.
Each CPT is considered as a Potential object in pyAgrum. There are several ways to fill such an object.<br/>

To get the CPT of a variable, use the cpt method of your BayesNet instance with the variable's id as parameter.<br/>

Now we are ready to fill in the parameters of each node in our network. 

In [6]:
bn.cpt(s).fillWith([0.5,0.5])

(pyAgrum.Potential<double>@00000252AF6910D0) 
  Smoking          |
no       |yes      |
---------|---------|
 0.5000  | 0.5000  |

In [7]:
bn.cpt(c)[:]=[ [0.99,0.01],[0.9,0.1]]
bn.cpt(c)

(pyAgrum.Potential<double>@00000252AAC3C630) 
      ||  Lung_Cancer      |
Smokin||no       |yes      |
------||---------|---------|
no    || 0.9900  | 0.0100  |
yes   || 0.9000  | 0.1000  |

In [8]:
bn.cpt(b)[:]=[ [0.7,0.3],[0.4,0.6]]
bn.cpt(b)

(pyAgrum.Potential<double>@00000252AAC3C850) 
      ||  Bronchitis       |
Smokin||no       |yes      |
------||---------|---------|
no    || 0.7000  | 0.3000  |
yes   || 0.4000  | 0.6000  |

In [9]:
bn.cpt(d)[{'Lung_Cancer': 0, 'Bronchitis': 0}] = [0.9, 0.1] 
bn.cpt(d)[{'Lung_Cancer': 0, 'Bronchitis': 1}] = [0.2, 0.8] 
bn.cpt(d)[{'Lung_Cancer': 1, 'Bronchitis': 0}] = [0.3, 0.7] 
bn.cpt(d)[{'Lung_Cancer': 1, 'Bronchitis': 1}] = [0.1, 0.9] 
bn.cpt(d)

(pyAgrum.Potential<double>@00000252AAC3C930) 
             ||  Dyspnoea         |
Bronch|Lung_C||no       |yes      |
------|------||---------|---------|
no    |no    || 0.9000  | 0.1000  |
yes   |no    || 0.2000  | 0.8000  |
no    |yes   || 0.3000  | 0.7000  |
yes   |yes   || 0.1000  | 0.9000  |

### Inference

We have to choose an inference engine to perform calculations for us. Many inference engines are currently available in pyAgrum:

- **Exact inference**, particularly :
    - `gum.LazyPropagation` : an exact inference method that transforms the Bayesian network into a hypergraph called a join tree or a junction tree. This tree is constructed in order to optimize inference computations.
    - others: `gum.VariableElimination`, `gum.ShaferShenoy`, ...
        
- **Samplig Inference** : approximate inference engine using sampling algorithms to generate a sequence of samples from the joint probability distribution (`gum.GibbSSampling`, etc.)
- **Loopy Belief Propagation** : approximate inference engine using inference algorithm exact for trees but not for DAG

In [10]:
ie=gum.LazyPropagation(bn)



scenario 1: we know that patient was a smoker, see the likelihood of lung cancer and bronchitis

scenario 2: we also observe that no dysponea

In [11]:
ie.setEvidence({'Smoking':'yes'})
ie.makeInference()
ie.posterior(c)

(pyAgrum.Potential<double>@00000252AFA4A7A0) 
  Lung_Cancer      |
no       |yes      |
---------|---------|
 0.9000  | 0.1000  |

In [12]:
ie.posterior(b)

(pyAgrum.Potential<double>@00000252AFA4A480) 
  Bronchitis       |
no       |yes      |
---------|---------|
 0.4000  | 0.6000  |

In [13]:
ie.setEvidence({'Smoking':'yes', 'Dyspnoea':'no'})
ie.makeInference()
ie.posterior(c)

(pyAgrum.Potential<double>@00000252AFA4AA80) 
  Lung_Cancer      |
no       |yes      |
---------|---------|
 0.9600  | 0.0400  |

In [14]:
ie.posterior(b)

(pyAgrum.Potential<double>@00000252AFA4A640) 
  Bronchitis       |
no       |yes      |
---------|---------|
 0.7467  | 0.2533  |

### Save BN

Now our BN is complete. It can be saved in different format :
bif|dsl|net|bifxml|o3prm|uai|xdsl

In [15]:
gum.saveBN(bn,"Demo.net")